In [46]:
import pandas as pd
import numpy as np

In [2]:
structure = pd.read_csv("kaggle/structure.csv")
targets = pd.read_csv("kaggle/targets.csv")
df = pd.read_csv("kaggle/user_activity.csv")

In [3]:
costed_steps = structure[structure.step_cost > 0]

In [4]:
def f(x):
    passed = x[x.action == "passed"]
#     print(passed)
    two_weeks = 10*3600*24
    border = passed.time.max() - two_weeks
    
    return len([1 for t in passed.time if t >= border])

In [35]:
from tqdm import tqdm
tqdm.pandas()

X = df[df.step_id.isin(costed_steps.step_id)].groupby("user_id").progress_apply(f)
Y = targets[targets.user_id.isin(X.index)]

100%|██████████| 9426/9426 [00:09<00:00, 1006.04it/s]


In [28]:
X.values, Y.passed.values

(array([0, 9, 9, ..., 2, 0, 3]), array([0, 0, 0, ..., 0, 0, 0]))

In [65]:
X_train = X.values.reshape(-1, 1)
Y_train = Y.passed.values.reshape(-1, 1)

In [66]:
from sklearn.linear_model import SGDClassifier
classifier = SGDClassifier()

In [67]:
X_train.shape, Y_train.shape

((9425, 1), (9425, 1))

In [69]:
classifier.fit(X_train, Y_train)

/Library/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [110]:
def prepare(df):
    X = df.groupby("user_id").progress_apply(f)
    return X.index, X.values.reshape(len(X), 1)

In [111]:
user_ids, X_test = prepare(pd.read_csv("kaggle/user_activity_test.csv"))
predicted = classifier.predict(X_test)

100%|██████████| 4256/4256 [00:04<00:00, 1022.17it/s]


In [114]:
res = pd.DataFrame({"user_id": user_ids, "passed": predicted}).set_index("user_id")

In [116]:
res.to_csv("result.csv")